<a href="https://colab.research.google.com/github/davideneffat/movie_recommendations/blob/main/movielens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import torch
from torch import Tensor
print(torch.__version__)

2.1.0+cu121


In [107]:
# Install required packages.
import os
os.environ['TORCH'] = torch.__version__

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
Looking in links: https://data.pyg.org/whl/nightly/torch-2.1.0+cu121.html
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-ao77r0qs
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-ao77r0qs
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit f757e72b02fefe20e710b3be50b8c4ebe585db7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [108]:
from torch_geometric.data import download_url, extract_zip

url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

movies_path = './ml-latest-small/movies.csv'
ratings_path = './ml-latest-small/ratings.csv'

Using existing file ml-latest-small.zip
Extracting ./ml-latest-small.zip


In [109]:
import pandas as pd

print('movies.csv:')
pd.read_csv(movies_path).head()

movies.csv:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [110]:
movies_df = pd.read_csv(movies_path, index_col='movieId')

# Split genres and convert into indicator variables:
genres = movies_df['genres'].str.get_dummies('|')#creates a binary indicator for each genre, separated by the '|' delimiter
print(genres[["Action", "Adventure", "Drama", "Horror","Animation","Fantasy"]].head())

# Use genres as movie input features:
movie_feat = torch.from_numpy(genres.values).to(torch.float)#tensor of movie features, size (9742, 20) = (# nodes, # genres)

         Action  Adventure  Drama  Horror  Animation  Fantasy
movieId                                                      
1             0          1      0       0          1        1
2             0          1      0       0          0        1
3             0          0      0       0          0        0
4             0          0      1       0          0        0
5             0          0      0       0          0        0


In [111]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [112]:
ratings_df = pd.read_csv(ratings_path)

#to take into account review rating: duplicate each row (so each edge) for a value equal to the rating
ratings_df = ratings_df.loc[ratings_df.index.repeat(ratings_df['rating'])].reset_index(drop=True)
ratings_df.head(15)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,1,4.0,964982703
2,1,1,4.0,964982703
3,1,1,4.0,964982703
4,1,3,4.0,964981247
5,1,3,4.0,964981247
6,1,3,4.0,964981247
7,1,3,4.0,964981247
8,1,6,4.0,964982224
9,1,6,4.0,964982224


In [113]:
# Create a mapping from unique user indices to range [0, num_user_nodes):
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedID': pd.RangeIndex(len(unique_user_id)),#because userId are not consecutive values, there are jumps. Same for movies
})
print("Mapping of user IDs to consecutive values:")
print(unique_user_id.head())
print(unique_user_id.tail())
print()
# Create a mapping from unique movie indices to range [0, num_movie_nodes):
unique_movie_id = ratings_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': movies_df.index,
    'mappedID': pd.RangeIndex(len(movies_df)),
})
print("Mapping of movie IDs to consecutive values:")
print(unique_movie_id.head())
print(unique_movie_id.tail())

# Perform merge to obtain the edges from users and movies:
ratings_user_id = pd.merge(ratings_df['userId'], unique_user_id, left_on='userId', right_on='userId', how='left')
ratings_user_id = torch.from_numpy(ratings_user_id['mappedID'].values)#create tensor of user ids
ratings_movie_id = pd.merge(ratings_df['movieId'], unique_movie_id, left_on='movieId', right_on='movieId', how='left')
ratings_movie_id = torch.from_numpy(ratings_movie_id['mappedID'].values)#create tensor of movies ids

#create tensor of edges (userId -> movieId)
edge_index_user_to_movie = torch.stack([ratings_user_id, ratings_movie_id], dim=0)#joint the 2 id tensors into a single tensor of size (2, #edges)

print()
print("Final edge indices pointing from users to movies:")
print(edge_index_user_to_movie)

Mapping of user IDs to consecutive values:
   userId  mappedID
0       1         0
1       2         1
2       3         2
3       4         3
4       5         4
     userId  mappedID
605     606       605
606     607       606
607     608       607
608     609       608
609     610       609

Mapping of movie IDs to consecutive values:
   movieId  mappedID
0        1         0
1        2         1
2        3         2
3        4         3
4        5         4
      movieId  mappedID
9737   193581      9737
9738   193583      9738
9739   193585      9739
9740   193587      9740
9741   193609      9741

Final edge indices pointing from users to movies:
tensor([[   0,    0,    0,  ...,  609,  609,  609],
        [   0,    0,    0,  ..., 9503, 9503, 9503]])


In [114]:
#define functions to get original movie id from the mapped one and viceversa
def get_original_node(index):
  row = unique_movie_id[unique_movie_id['mappedID'] == index]
  return row['movieId'].values[0]

def get_mapped_node(index):
  row = unique_movie_id[unique_movie_id['movieId'] == index]
  return row['mappedID'].values[0]

print(get_original_node(6405))
print(get_mapped_node(1258))

50872
957


In [115]:
#let's create the graph
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()

# Save node indices:
data["user"].node_id = torch.arange(len(unique_user_id))
data["movie"].node_id = torch.arange(len(movies_df))

# Add the node features and edge indices:
data["movie"].x = movie_feat
data["user", "rates", "movie"].edge_index = edge_index_user_to_movie  #edge_index = tensor (2,#edges) with for each edge (mapped) user id and (mapped) movie id

# We also need to make sure to add the reverse edges from movies to users
# in order to let a GNN be able to pass messages in both directions.
# We can leverage the `T.ToUndirected()` transform for this from PyG:
data = T.ToUndirected()(data)

print(data)


HeteroData(
  user={ node_id=[610] },
  movie={
    node_id=[9742],
    x=[9742, 20],
  },
  (user, rates, movie)={ edge_index=[2, 337884] },
  (movie, rev_rates, user)={ edge_index=[2, 337884] }
)


In [116]:
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 70% of edges for message passing, and 30% of edges for supervision.
transform = T.RandomLinkSplit(
    num_val=0.1,#10% validation set
    num_test=0.1,#10% test set
    disjoint_train_ratio=0.3,#30% disjoint training set
    neg_sampling_ratio=2.0,#negative samples are double of positive samples
    add_negative_train_samples=False,
    edge_types=("user", "rates", "movie"),
    rev_edge_types=("movie", "rev_rates", "user"),
)

# edge_index (edges used for message passing) + edge_label_index (edges used for supervision) = training_set = 80%

train_data, val_data, test_data = transform(data)
print("Training data:")
print("--------------")
print(train_data)
print()
print("User IDs:")
print(train_data["user", "rates", "movie"].edge_index[0][:10])
print("Movie IDs:")
print(train_data["user", "rates", "movie"].edge_index[1][:10])
print("Edges labels:")
print(train_data["user", "rates", "movie"].edge_label[:10])
print(train_data["user", "rates", "movie"].edge_label[-10:]) #only positive labels (1)
print("Edges labels indices:")
print(train_data["user", "rates", "movie"].edge_label_index[0][:10])
print(train_data["user", "rates", "movie"].edge_label_index[1][:10])

print()
print("Validation data:")
print("-----------------")
print(val_data)
print()
print("Test data:")
print("-----------")
print(test_data)

Training data:
--------------
HeteroData(
  user={ node_id=[610] },
  movie={
    node_id=[9742],
    x=[9742, 20],
  },
  (user, rates, movie)={
    edge_index=[2, 189216],
    edge_label=[81092],
    edge_label_index=[2, 81092],
  },
  (movie, rev_rates, user)={ edge_index=[2, 189216] }
)

User IDs:
tensor([112, 273, 598, 297, 578, 384, 596,  27,  56, 121])
Movie IDs:
tensor([1541, 1033, 2571, 7980,   16,  513, 1797, 6156, 1702, 7605])
Edges labels:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Edges labels indices:
tensor([390, 605, 574, 605, 476, 304,  19,  61,  40, 114])
tensor([3628, 4566, 1937, 4893, 3783,  508, 1563, 3574, 6657,  328])

Validation data:
-----------------
HeteroData(
  user={ node_id=[610] },
  movie={
    node_id=[9742],
    x=[9742, 20],
  },
  (user, rates, movie)={
    edge_index=[2, 270308],
    edge_label=[101364],
    edge_label_index=[2, 101364],
  },
  (movie, rev_rates, user)={ edge_index=[2, 270308] 

In [117]:
#creation of mini batches
from torch_geometric.loader import LinkNeighborLoader

#The data loader will add the following attributes to the returned mini-batch:
#-n_id The global node index for every sampled node
#-e_id The global edge index for every sampled edge
#-input_id: The global index of the edge_label_index
#-num_sampled_nodes: The number of sampled nodes in each hop
#-num_sampled_edges: The number of sampled edges in each hop

# Define seed edges:
edge_label_index = train_data["user", "rates", "movie"].edge_label_index  #Size([2, 24201])
edge_label = train_data["user", "rates", "movie"].edge_label  #Size([24201])

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],#The number of neighbors to sample for each node in each iteration (20 the first, 10 the second)
    neg_sampling_ratio=2.0,
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    edge_label=edge_label,#The labels of edge indices for which neighbors are sampled
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "movie"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "rates", "movie"].edge_label.min() == 0
assert sampled_data["user", "rates", "movie"].edge_label.max() == 1

Sampled mini-batch:
HeteroData(
  user={
    node_id=[606],
    n_id=[606],
    num_sampled_nodes=[3],
  },
  movie={
    node_id=[2607],
    x=[2607, 20],
    n_id=[2607],
    num_sampled_nodes=[3],
  },
  (user, rates, movie)={
    edge_index=[2, 20842],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[20842],
    num_sampled_edges=[2],
    input_id=[128],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 8030],
    e_id=[8030],
    num_sampled_edges=[2],
  }
)


In [118]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions: (Combines user and movie node embeddings to create edge-level representations)
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_movie).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        self.movie_lin = torch.nn.Linear(20, hidden_channels)  #(movie_lin): Linear(in_features=20, out_features=64, bias=True)
        self.user_emb = torch.nn.Embedding(data["user"].num_nodes, hidden_channels)#learn node embedding for users
        self.movie_emb = torch.nn.Embedding(data["movie"].num_nodes, hidden_channels)#learn node embedding for movies

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant (we have user and movie types nodes):
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "user": self.user_emb(data["user"].node_id),#just learned embeddings
          "movie": self.movie_lin(data["movie"].x) + self.movie_emb(data["movie"].node_id),#movie genres + learned embeddings
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["movie"],
            data["user", "rates", "movie"].edge_label_index,
        )

        return pred


model = Model(hidden_channels=64)

print(model)

Model(
  (movie_lin): Linear(in_features=20, out_features=64, bias=True)
  (user_emb): Embedding(610, 64)
  (movie_emb): Embedding(9742, 64)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__rates__movie): SAGEConv(64, 64, aggr=mean)
      (movie__rev_rates__user): SAGEConv(64, 64, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__rates__movie): SAGEConv(64, 64, aggr=mean)
      (movie__rev_rates__user): SAGEConv(64, 64, aggr=mean)
    )
  )
  (classifier): Classifier()
)


In [119]:
#train model
import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 11):#reduce to 6 for 10/20m dataset
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):#iterating over mini-batches
        optimizer.zero_grad()

        sampled_data.to(device)
        pred = model(sampled_data)

        ground_truth = sampled_data["user", "rates", "movie"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cpu'


100%|██████████| 634/634 [00:43<00:00, 14.71it/s]


Epoch: 001, Loss: 0.3481


100%|██████████| 634/634 [00:28<00:00, 22.48it/s]


Epoch: 002, Loss: 0.2613


...

Epoch: 008, Loss: 0.1943


100%|██████████| 634/634 [00:29<00:00, 21.77it/s]


Epoch: 009, Loss: 0.1888


100%|██████████| 634/634 [00:31<00:00, 20.18it/s]

Epoch: 010, Loss: 0.1851


In [120]:
# Define the validation seed edges:
edge_label_index = val_data["user", "rates", "movie"].edge_label_index
edge_label = val_data["user", "rates", "movie"].edge_label

val_loader = LinkNeighborLoader(#creates mini-batches for validation set
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  user={
    node_id=[607],
    n_id=[607],
    num_sampled_nodes=[3],
  },
  movie={
    node_id=[2537],
    x=[2537, 20],
    n_id=[2537],
    num_sampled_nodes=[3],
  },
  (user, rates, movie)={
    edge_index=[2, 21315],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[21315],
    num_sampled_edges=[2],
    input_id=[384],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 7720],
    e_id=[7720],
    num_sampled_edges=[2],
  }
)


In [121]:
from sklearn.metrics import roc_auc_score

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))
        ground_truths.append(sampled_data["user", "rates", "movie"].edge_label)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")
print()
print("Predictions for existing edges:")
print(pred[:30])
print(ground_truth[:30])
print("Predictions for non-existing edges:")
print(pred[-30:])
print(ground_truth[-30:])

100%|██████████| 264/264 [00:10<00:00, 24.64it/s]



Validation AUC: 0.9764

Predictions for existing edges:
[ 2.4669647   2.4979408   0.81848854  1.5462525   2.2795901   9.55509
  2.9646082   2.7604096   5.2873087   1.3714457   4.101929   -0.47849905
  2.5352364   0.8687235   0.9914876   5.0881557   1.8479302   3.8725715
  1.7306535   3.8996124   3.235597    8.031215   -3.972323    0.13711715
  3.3931692   1.7584181   1.3292202   3.4456224  -0.34229046  2.4030685 ]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
Predictions for non-existing edges:
[ -1.8952775 -12.332608  -15.459254  -11.31727    -6.9593344 -18.27707
  -1.5983788  -8.888111   -7.513619   -6.023022  -12.099562  -13.766905
  -1.7206643 -11.803905  -10.981283  -18.920673   -7.373623  -16.659634
 -13.001235   -2.0698514  -5.753574   -8.805517   -5.1431074 -13.584836
  -2.5608237  -3.5198462  -4.03349    -6.054299    0.573362   -6.098725 ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.

In [122]:
#test our model prediction for a specific user:
user_index = 1  #value from 1 to 610 for the 100k nodes dataset

In [123]:
user_index = user_index - 1  #due to the mapping (mapped used ids start from 0)
user_ids = torch.full((data["movie"].num_nodes,), user_index)
movie_ids = torch.tensor(unique_movie_id['mappedID'])
edge_label_index = torch.stack([user_ids, movie_ids], dim=0)#joint the 2 id tensors into a single tensor of size (2, #edges)

print(edge_label_index)  #tensor with all possible movies that our user can see

val_loader = LinkNeighborLoader(#creates mini-batches for validation set
    data=data,
    num_neighbors=[20, 10],
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    batch_size=3 * 128,
    shuffle=False,
)

preds = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))

pred = torch.cat(preds, dim=0).cpu().numpy()

print()

tensor([[   0,    0,    0,  ...,    0,    0,    0],
        [   0,    1,    2,  ..., 9739, 9740, 9741]])


100%|██████████| 26/26 [00:00<00:00, 35.62it/s]

In [124]:
def get_max(): #function that extract max probability from probabilities array
  max = pred[0]
  #maxIndex = 0
  for i in range(len(pred)):
      if pred[i] > max:
          max = pred[i]
          maxIndex = i
  print(get_original_node(maxIndex))#already seen movies, the last is the not-seen one
  return maxIndex

In [125]:
user_index = user_index + 1  #reverse mapping

In [126]:
#let's find the first most similar movie not already seen
most_similar_movie = get_max()
row = ratings_df[(ratings_df['userId'] == user_index) & (ratings_df['movieId'] == get_original_node(most_similar_movie))]
while not row.empty: #repeat untill find a not-seen movie
  pred[most_similar_movie] = -100
  most_similar_movie = get_max()
  row = ratings_df[(ratings_df['userId'] == user_index) & (ratings_df['movieId'] == get_original_node(most_similar_movie))]
print("Most similar not-seen movie for user {} is {}".format(user_index, get_original_node(most_similar_movie)))

1210
260
2081
Most similar not-seen movie for user 1 is 2081


In [127]:
#let's show the movies that our user has seen
print("Movies already seen by user {}:".format(user_index))
result_df = ratings_df[ratings_df['userId'] == user_index].merge(movies_df, on='movieId', how='inner')
unique_movies = result_df.drop_duplicates(subset=['movieId'])
unique_movies[['rating','title','genres']]

Movies already seen by user 1:


,rating,title,genres
0,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,4.0,Grumpier Old Men (1995),Comedy|Romance
8,4.0,Heat (1995),Action|Crime|Thriller
12,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
17,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...
991,4.0,Shaft (2000),Action|Crime|Thriller
995,5.0,X-Men (2000),Action|Adventure|Sci-Fi
1000,4.0,What About Bob? (1991),Comedy
1004,4.0,Transformers: The Movie (1986),Adventure|Animation|Children|Sci-Fi


In [128]:
#let's print our prediction
print("Recommended movie for user {}:".format(user_index))
print("'{}'".format(movies_df[movies_df.index == get_original_node(most_similar_movie)]['title'].values[0]))
print("It's genres are:")
print(movies_df[movies_df.index == get_original_node(most_similar_movie)]['genres'].values)

Recommended movie for user 1:
'Little Mermaid, The (1989)'
It's genres are:
['Animation|Children|Comedy|Musical|Romance']
